# 4. Comparing training runs and Hyperparameter (HP) tuning

In this chapter, you will direct your attention towards the analysis of model performance and the fine-tuning of hyperparameters. You will acquire practical expertise in comparing metrics and visualizations across different branches to assess changes in model performance. You will conduct hyperparameter tuning using scikit-learn's GridSearchCV. Furthermore, you will delve into the automation of pull requests using the optimal model configuration.

## Documentation

- [Convert a HTML page to Markdown in GitHub Actions](https://github.com/marketplace/actions/convert-html-to-markdown)
- [GitHub Actions: how to target all branches EXCEPT master?](https://stackoverflow.com/questions/57699839/github-actions-how-to-target-all-branches-except-master)
- [Github Actions - trigger another action after one action is completed](https://stackoverflow.com/questions/62750603/github-actions-trigger-another-action-after-one-action-is-completed)
- [How to Stop Tracking Data](https://dvc.org/doc/user-guide/how-to/stop-tracking-data)

## 4.1 Comparing metrics and plots in DVC

### Preparing the environment

In [1]:
# Moving to myfeature
!git checkout myfeature
!git pull origin myfeature

Already on 'myfeature'


Already up to date.


From https://github.com/jacesca/CICD-Workflow
 * branch            myfeature  -> FETCH_HEAD


In [2]:
# Removing files from git versioning
# To ensure this files can be tracked in dvc.
!git add ml-example3\processed-data\weather.csv
!git add ml-example3\evaluation-result\confusion_matrix.png
!git commit -m "tracking ml-ex3 .. weather.cs and confusion_matrix.png"
!git rm -r --cached ml-example3\processed-data\weather.csv
!git rm -r --cached ml-example3\evaluation-result\confusion_matrix.png
!git commit -m "stop tracking ml-ex3 .. weather.cs and confusion_matrix.png"

The following paths are ignored by one of your .gitignore files:
ml-example3/processed-data/weather.csv
hint: Use -f if you really want to add them.
hint: Disable this message with "git config advice.addIgnoredFile false"
The following paths are ignored by one of your .gitignore files:
ml-example3/evaluation-result/confusion_matrix.png
hint: Use -f if you really want to add them.
hint: Disable this message with "git config advice.addIgnoredFile false"


On branch myfeature
nothing to commit, working tree clean


fatal: pathspec 'ml-example3\processed-data\weather.csv' did not match any files
fatal: pathspec 'ml-example3\evaluation-result\confusion_matrix.png' did not match any files


On branch myfeature
nothing to commit, working tree clean


In [3]:
# Ensuring RFC_FOREST_DEPTH=2
with open("ml-example3/.env", "w") as f:
    f.write('RFC_FOREST_DEPTH=2\n')

In [4]:
# Removing any previous `dvc.yaml` file
# Removing any existing previous yml file
!del dvc.yaml

### Running the process manually

In [5]:
!python .\ml-example3\preprocess_data.py
!python .\ml-example3\train.py

====================Test Set Metrics==================
{
  "accuracy": 0.947,
  "precision": 0.988,
  "recall": 0.7702,
  "f1_score": 0.8656
}


### Working with DVC pipeline for `ml-example3`

In [6]:
# Prepare the yaml file
yml_str = f"""
stages:
  preprocess:
    cmd: python ml-example3/preprocess_data.py
    deps:
    - ml-example3/raw-data/weather.csv
    - ml-example3/preprocess_data.py
    - ml-example3/.env
    - ml-example3/utils_and_constants.py
    outs:
    - ml-example3/processed-data/weather.csv
  train:
    cmd: python ml-example3/train.py
    deps:
    - ml-example3/metrics_and_plots.py
    - ml-example3/model.py
    - ml-example3/processed-data/weather.csv
    - ml-example3/train.py
    - ml-example3/.env
    - ml-example3/utils_and_constants.py
    outs:
    - ml-example3/evaluation-result/confusion_matrix.png
    metrics:
      - ml-example3/evaluation-result/metrics.json:
          cache: false
    plots:
      - ml-example3/evaluation-result/predictions.csv:
          template: confusion_normalized
          x: predicted_label
          y: true_label
          x_label: 'Predicted label'
          y_label: 'True label'
          title: Confusion matrix
          cache: false
"""
with open("dvc.yaml", "w") as f:
    f.write(yml_str)

In [7]:
# Reviewing the dvc.yaml file
!more dvc.yaml


stages:
  preprocess:
    cmd: python ml-example3/preprocess_data.py
    deps:
    - ml-example3/raw-data/weather.csv
    - ml-example3/preprocess_data.py
    - ml-example3/.env
    - ml-example3/utils_and_constants.py
    outs:
    - ml-example3/processed-data/weather.csv
  train:
    cmd: python ml-example3/train.py
    deps:
    - ml-example3/metrics_and_plots.py
    - ml-example3/model.py
    - ml-example3/processed-data/weather.csv
    - ml-example3/train.py
    - ml-example3/.env
    - ml-example3/utils_and_constants.py
    outs:
    - ml-example3/evaluation-result/confusion_matrix.png
    metrics:
      - ml-example3/evaluation-result/metrics.json:
          cache: false
    plots:
      - ml-example3/evaluation-result/predictions.csv:
          template: confusion_normalized
          x: predicted_label
          y: true_label
          x_label: 'Predicted label'
          y_label: 'True label'
          title: Confusion matrix
          cache: false


In [8]:
# Reviewing the process
!dvc dag

+------------+ 
| preprocess | 
+------------+ 
       *       
       *       
       *       
  +-------+    
  | train |    
  +-------+    


In [9]:
# Reviewing the comands that the pipeline will execute:
!dvc repro --dry

Stage 'preprocess' is cached - skipping run, checking out outputs
Running stage 'preprocess':
> python ml-example3/preprocess_data.py

Stage 'train' is cached - skipping run, checking out outputs
Running stage 'train':
> python ml-example3/train.py
Use `dvc push` to send your updates to remote storage.


In [10]:
# Running the dvc pipeline
!dvc repro

Stage 'preprocess' is cached - skipping run, checking out outputs
Updating lock file 'dvc.lock'

Stage 'train' is cached - skipping run, checking out outputs
Updating lock file 'dvc.lock'

To track the changes with git, run:

	git add dvc.lock

To enable auto staging, run:

	dvc config core.autostage true
Use `dvc push` to send your updates to remote storage.


### Querying and comparing DVC metrics

In [11]:
!dvc metrics show

Path                                        accuracy    f1_score    precision    recall
ml-example3\evaluation-result\metrics.json  0.947       0.8656      0.988        0.7702


In [12]:
!dvc metrics show --md

| Path                                       | accuracy   | f1_score   | precision   | recall   |
|--------------------------------------------|------------|------------|-------------|----------|
| ml-example3\evaluation-result\metrics.json | 0.947      | 0.8656     | 0.988       | 0.7702   |



In [13]:
!dvc metrics diff --all

Path                                        Metric     HEAD    workspace    Change
ml-example3\evaluation-result\metrics.json  accuracy   0.9912  0.947        -0.0442
ml-example3\evaluation-result\metrics.json  f1_score   0.98    0.8656       -0.1144
ml-example3\evaluation-result\metrics.json  precision  0.9861  0.988        0.0019
ml-example3\evaluation-result\metrics.json  recall     0.974   0.7702       -0.2038


In [14]:
# Compare metrics with main branch
!git fetch --prune
!dvc metrics diff HEAD --all

Path                                        Metric     HEAD    workspace    Change
ml-example3\evaluation-result\metrics.json  accuracy   0.9912  0.947        -0.0442
ml-example3\evaluation-result\metrics.json  f1_score   0.98    0.8656       -0.1144
ml-example3\evaluation-result\metrics.json  precision  0.9861  0.988        0.0019
ml-example3\evaluation-result\metrics.json  recall     0.974   0.7702       -0.2038


### Commiting changes in myfeature branch

In [15]:
!dvc push
!git add .
!git commit -m "DVC Pipeline for ml-example3"
!git push origin myfeature

Everything is up to date.
[myfeature 4d62ff9] DVC Pipeline for ml-example3
 5 files changed, 545 insertions(+), 835 deletions(-)


To https://github.com/jacesca/CICD-Workflow.git
   3029c5a..4d62ff9  myfeature -> myfeature


### Setting up DVC Github Action

```
name: comments-ml-example3

on:
  pull_request:
    branches: ["master", "myfeature"]

permissions: write-all

jobs:
  train_and_test_model_ml3:
    runs-on: ubuntu-latest
    steps:
      - name: Checkout
        uses: actions/checkout@v4

      - name: Setup Python
        uses: actions/setup-python@v5
        with:
          python-version: 3.11.9

      - name: Install Dependencies
        run: |
          pip install -r requirements.txt

      - name: Train and test model
        run: |
          python ml-example3/preprocess_data.py
          python ml-example3/train.py

      - name: Create DVC pipeline
        run: |
          python ml-example3/dvc_pipeline_creation.py

      - name: Setup DVC
        uses: iterative/setup-dvc@v1

      - name: Run DVC pipeline
        run: dvc repro
        
      - name: Setup CML GitHub Actions
        uses: iterative/setup-cml@v3

      - name: Write CML report
        env:
          REPO_TOKEN: ${{secrets.GITHUB_TOKEN}}
        run: |
          echo "# ML Example 3" > report3.md
          dvc metrics show --md >> report3.md
          echo "### Metrics comparison with master branch" >> report3.md
          dvc metrics diff --all --md >> report3.md
          echo "![Confusion Matrix Plot](ml-example3/evaluation-result/confusion_matrix.png)" >> report3.md
          cml comment create report3.md

```

### Plotting Confusion Matrix

In [16]:
# Plotting what was configured
!dvc plots show

file:///C:/Users/Jacqueline/Documents/projects/CAMP-MLEngTrack/14-CICD%20for%20Machine%20Learning/dvc_plots/index.html


In [17]:
from IPython.display import IFrame
IFrame("dvc_plots/index.html", width=800, height=450)

In [18]:
# Plotting other graphs
# scatter - scatter plot
# linear - interactive linear plot
# simple - non-interactive customizable linear plot
# smooth - linear plot with smoothing
# confusion - confusion matrix
# confusion_normalized - confusion matrix with values normalized to <0, 1> range
# bar_horizontal - horizontal bar plot
# bar_horizontal_sorted - horizontal bar plot sorted by bar size
!dvc plots show ml-example3/evaluation-result/predictions.csv \
                -t confusion \
                -x predicted_label \
                -y true_label \
                --x-label "Predicted Label" \
                --y-label "True label" \
                --title "Confusion matrix"

file:///C:/Users/Jacqueline/Documents/projects/CAMP-MLEngTrack/14-CICD%20for%20Machine%20Learning/dvc_plots/index.html


In [19]:
from IPython.display import IFrame
IFrame("dvc_plots/index.html", width=800, height=450)

### Comparing Confusion Matrix

In [20]:
!dvc plots diff master

file:///C:/Users/Jacqueline/Documents/projects/CAMP-MLEngTrack/14-CICD%20for%20Machine%20Learning/dvc_plots/index.html


In [21]:
from IPython.display import IFrame
IFrame("dvc_plots/index.html", width=800, height=450)

### Ex.1 - Adding metrics and plots to dvc.yaml
In this exercise, your task is to complete the contents of dvc.yaml that defines a model training workflow.

Here preprocess_dataset.py and train.py are the files that perform data preprocessing and model training by taking weather.csv as input in the raw_dataset folder. As output, the model training code generates a predictions.csv file that contains the predictions and the ground truth, and metrics.json file containing structured metrics data. The former would be used to generate a normalized confusion matrix plot for comparing it with previous commits.

**Instruction**
1. Set the metrics target to the output metrics file.
2. Set the plot target to the output file containing predictions data.
3. Set the plot template to confusion_normalized to plot the normalized confusion matrix.
4. Set the correct value for cache key to track plots in Git repository instead of DVC remote.

In [22]:
!more dvc.yaml


stages:
  preprocess:
    cmd: python ml-example3/preprocess_data.py
    deps:
    - ml-example3/raw-data/weather.csv
    - ml-example3/preprocess_data.py
    - ml-example3/.env
    - ml-example3/utils_and_constants.py
    outs:
    - ml-example3/processed-data/weather.csv
  train:
    cmd: python ml-example3/train.py
    deps:
    - ml-example3/metrics_and_plots.py
    - ml-example3/model.py
    - ml-example3/processed-data/weather.csv
    - ml-example3/train.py
    - ml-example3/.env
    - ml-example3/utils_and_constants.py
    outs:
    - ml-example3/evaluation-result/confusion_matrix.png
    metrics:
      - ml-example3/evaluation-result/metrics.json:
          cache: false
    plots:
      - ml-example3/evaluation-result/predictions.csv:
          template: confusion_normalized
          x: predicted_label
          y: true_label
          x_label: 'Predicted label'
          y_label: 'True label'
          title: Confusion matrix
          cache: false


### Ex.2 - Comparing metrics across Git branches

In this exercise, you will use DVC for querying and comparing metrics across different branches. This functionality of DVC is helpful in making decisions about the quality of a machine learning model.

You will start in the main branch, where a DVC pipeline has already been executed and results committed in Git. Your task would entail querying metrics in the main branch. Then, you'll switch to a new training branch, change a hyperparameter, and execute the pipeline again, followed by comparing metrics with the main branch.

**Instruction:**

1. Query the metrics in the master branch by `running dvc metrics` show command in the terminal.
2. Change `RFC_FOREST_DEPTH` to `4`, and execute the DVC pipeline.
4. Compare the changed metrics with the master branch using `dvc metrics diff --md | tee metrics_diff.md` command.

In [23]:
# Comitting any missing change
!dvc push
!git add .
!git commit -m "Metrics changes"
!git push origin myfeature

Everything is up to date.
On branch myfeature
nothing to commit, working tree clean


Everything up-to-date


In [24]:
# 1. Query the metrics in the branch
!dvc metrics show

Path                                        accuracy    f1_score    precision    recall
ml-example3\evaluation-result\metrics.json  0.947       0.8656      0.988        0.7702


In [25]:
# 2. Change `RFC_FOREST_DEPTH` to `4`
with open("ml-example3/.env", "w") as f:
    f.write('RFC_FOREST_DEPTH=4\n')

In [26]:
# and execute the DVC pipeline
!dvc repro --force

Running stage 'preprocess':
> python ml-example3/preprocess_data.py
Updating lock file 'dvc.lock'

Running stage 'train':
> python ml-example3/train.py
====================Test Set Metrics==================
{
  "accuracy": 0.9912,
  "precision": 0.9861,
  "recall": 0.974,
  "f1_score": 0.98
}
Updating lock file 'dvc.lock'

To track the changes with git, run:

	git add dvc.lock

To enable auto staging, run:

	dvc config core.autostage true
Use `dvc push` to send your updates to remote storage.


In [27]:
# Compare the changed metrics with the master branch
!dvc metrics diff --md --all

| Path                                       | Metric    | HEAD   | workspace   | Change   |
|--------------------------------------------|-----------|--------|-------------|----------|
| ml-example3\evaluation-result\metrics.json | accuracy  | 0.947  | 0.9912      | 0.0442   |
| ml-example3\evaluation-result\metrics.json | f1_score  | 0.8656 | 0.98        | 0.1144   |
| ml-example3\evaluation-result\metrics.json | precision | 0.988  | 0.9861      | -0.0019  |
| ml-example3\evaluation-result\metrics.json | recall    | 0.7702 | 0.974       | 0.2038   |



In [28]:
!dvc metrics diff --md --all > metrics_diff.md & type metrics_diff.md

| Path                                       | Metric    | HEAD   | workspace   | Change   |
|--------------------------------------------|-----------|--------|-------------|----------|
| ml-example3\evaluation-result\metrics.json | accuracy  | 0.947  | 0.9912      | 0.0442   |
| ml-example3\evaluation-result\metrics.json | f1_score  | 0.8656 | 0.98        | 0.1144   |
| ml-example3\evaluation-result\metrics.json | precision | 0.988  | 0.9861      | -0.0019  |
| ml-example3\evaluation-result\metrics.json | recall    | 0.7702 | 0.974       | 0.2038   |



In [29]:
!dir *.md /b

metrics_diff.md
More.md
Readme.md


### Ex.3 - Run DVC pipeline in GitHub Actions
In this exercise, you will use CML GitHub Action to run a DVC pipeline and compare metrics between the training branch and main. The pipeline will trigger when you open a PR against the main branch.

The output from running train.py is a metrics.json file containing model metrics that will provide the source data for comparing metrics across branches.

Your task is to finish the scaffolded .github/workflows/dvc_cml.yaml to formulate a high-level model training flow. Scroll down to Line 24 to make changes.

**Instruction**
    
1. Setup DVC GitHub Action iterative/setup-dvc@v1.
2. Run DVC pipeline in Run DVC pipeline step.
3. Compare metrics with main branch and write the markdown report in the Write CML report step.
4. Write the correct file in cml comment create command to create a comment in the PR.

In [30]:
!type .github\workflows\Comments-MLExample3V2.yml

name: comments-ml-example3-v2

on:
  pull_request:
    branches: ["master", "myfeature"]

permissions: write-all

jobs:
  train_and_test_model_ml3:
    runs-on: ubuntu-latest
    steps:
      - name: Checkout
        uses: actions/checkout@v4

      - name: Setup Python
        uses: actions/setup-python@v5
        with:
          python-version: 3.11.9

      - name: Install Dependencies
        run: |
          pip install -r requirements.txt

      - name: Train and test model
        run: |
          python ml-example2/preprocess_data.py
          python ml-example2/train.py

      - name: Setup DVC
        uses: iterative/setup-dvc@v1

      - name: Run DVC pipeline
        run: dvc repro
        
      - name: Setup CML GitHub Actions
        uses: iterative/setup-cml@v3

      - name: Write CML report
        env:
          REPO_TOKEN: ${{secrets.GITHUB_TOKEN}}
        run: |
          echo "# ML Example 3 (V.2)" > report3-2.md
          dvc metrics show --md >> report3-2.md
   

## 4.2 Hyperparameter Tuning with DVC

### Preparing the environment

In [85]:
# Removing any previous `dvc.yaml` file
# Removing any existing previous yml file
!del dvc.yaml

### Running the process manually

In [86]:
!python .\ml-example4\preprocess_dataset.py
!python .\ml-example4\hp_tuning.py
!python .\ml-example4\train.py

Fitting 5 folds for each of 9 candidates, totalling 45 fits
[CV] END ....max_depth=10, n_estimators=2, random_state=1993; total time=   0.0s
[CV] END ....max_depth=10, n_estimators=2, random_state=1993; total time=   0.0s
[CV] END ....max_depth=10, n_estimators=2, random_state=1993; total time=   0.0s
[CV] END ....max_depth=10, n_estimators=2, random_state=1993; total time=   0.0s
[CV] END ....max_depth=10, n_estimators=2, random_state=1993; total time=   0.0s
[CV] END ....max_depth=10, n_estimators=4, random_state=1993; total time=   0.0s
[CV] END ....max_depth=10, n_estimators=4, random_state=1993; total time=   0.0s
[CV] END ....max_depth=10, n_estimators=4, random_state=1993; total time=   0.0s
[CV] END ....max_depth=10, n_estimators=4, random_state=1993; total time=   0.0s
[CV] END ....max_depth=10, n_estimators=4, random_state=1993; total time=   0.0s
[CV] END ....max_depth=10, n_estimators=5, random_state=1993; total time=   0.0s
[CV] END ....max_depth=10, n_estimators=5, random

### Working with DVC pipeline for ml-example3

In [87]:
!python ml-example4/dvc_pipeline_creation.py

In [88]:
# Reviewing the dvc.yaml file
!more dvc.yaml

stages:
  preprocess:
    cmd: python ml-example4/preprocess_dataset.py
    deps:
    - ml-example4/raw-data/weather.csv
    - ml-example4/preprocess_dataset.py
    - ml-example4/.env
    - ml-example4/utils_and_constants.py
    outs:
    - ml-example4/processed-data/weather.csv
  hp_tune:
    cmd: python ml-example4/hp_tuning.py
    deps:
    - ml-example4/processed-data/weather.csv
    - ml-example4/config/hp_config.json
    - ml-example4/hp_tuning.py
    outs:
    - ml-example4/evaluation-result/rfc_best_params.json
    - ml-example4/evaluation-result/hp_tuning_results.md:
        cache: false
  train:
    cmd: python ml-example4/train.py
    deps:
    - ml-example4/metrics_and_plots.py
    - ml-example4/model.py
    - ml-example4/processed-data/weather.csv
    - ml-example4/evaluation-result/rfc_best_params.json
    - ml-example4/train.py
    - ml-example4/.env
    - ml-example4/utils_and_constants.py
    outs:
    - ml-example4/evaluation-result/confusion_matrix.png
    metrics:
 

In [89]:
# Reviewing the process
!dvc dag

        +------------+     
        | preprocess |     
        +------------+     
          **        **     
        **            **   
       *                ** 
+---------+               *
| hp_tune |             ** 
+---------+           **   
          **        **     
            **    **       
              *  *         
           +-------+       
           | train |       
           +-------+       


In [90]:
# Reviewing the comands that the pipeline will execute:
!dvc repro --dry --force

Running stage 'preprocess':
> python ml-example4/preprocess_dataset.py

Running stage 'hp_tune':
> python ml-example4/hp_tuning.py

Running stage 'train':
> python ml-example4/train.py
Use `dvc push` to send your updates to remote storage.


In [91]:
# Running the dvc pipeline
!dvc repro --force

Running stage 'preprocess':
> python ml-example4/preprocess_dataset.py

Running stage 'hp_tune':
> python ml-example4/hp_tuning.py
Fitting 5 folds for each of 9 candidates, totalling 45 fits
[CV] END ....max_depth=10, n_estimators=2, random_state=1993; total time=   0.0s
[CV] END ....max_depth=10, n_estimators=2, random_state=1993; total time=   0.0s
[CV] END ....max_depth=10, n_estimators=2, random_state=1993; total time=   0.0s
[CV] END ....max_depth=10, n_estimators=2, random_state=1993; total time=   0.0s
[CV] END ....max_depth=10, n_estimators=2, random_state=1993; total time=   0.0s
[CV] END ....max_depth=10, n_estimators=4, random_state=1993; total time=   0.0s
[CV] END ....max_depth=10, n_estimators=4, random_state=1993; total time=   0.0s
[CV] END ....max_depth=10, n_estimators=4, random_state=1993; total time=   0.0s
[CV] END ....max_depth=10, n_estimators=4, random_state=1993; total time=   0.0s
[CV] END ....max_depth=10, n_estimators=4, random_state=1993; total time=   0.0s

### Ex.4 - Adding Hyperparameter tuning to dvc.yaml
In this exercise, your task is to define a hyperparameter tuning workflow. The python file hp_tuning.py is the script for hyperparameter training and takes the hyperparameter configuration file hp_config.json as an input to produce rfc_best_params.json as an output.

The dvc.yaml file outlines the DVC workflow orchestrating the hyperparameter tuning and lists commands, dependencies, and outputs.

NOTE: This exercise involves changing both hp_tuning.py and dvc.yaml. Both files have been opened for you in the editor.

**Instruction:**
    
1. Set the hyperparameter tuning command running the python script hp_tuning.py in dvc.yaml.
2. Specify the hyperparameter configuration hp_config.json in dvc.yaml, and in hp_tuning.py.
3. Specify the hyperparameter training file hp_tuning.py in dvc.yaml as a dependency.
4. Perform Grid Search Cross Validation on training data in hp_tuning.py.

**Working with `MLExample4`**

In [92]:
# Reviewing the dvc.yaml file
!more dvc.yaml

stages:
  preprocess:
    cmd: python ml-example4/preprocess_dataset.py
    deps:
    - ml-example4/raw-data/weather.csv
    - ml-example4/preprocess_dataset.py
    - ml-example4/.env
    - ml-example4/utils_and_constants.py
    outs:
    - ml-example4/processed-data/weather.csv
  hp_tune:
    cmd: python ml-example4/hp_tuning.py
    deps:
    - ml-example4/processed-data/weather.csv
    - ml-example4/config/hp_config.json
    - ml-example4/hp_tuning.py
    outs:
    - ml-example4/evaluation-result/rfc_best_params.json
    - ml-example4/evaluation-result/hp_tuning_results.md:
        cache: false
  train:
    cmd: python ml-example4/train.py
    deps:
    - ml-example4/metrics_and_plots.py
    - ml-example4/model.py
    - ml-example4/processed-data/weather.csv
    - ml-example4/evaluation-result/rfc_best_params.json
    - ml-example4/train.py
    - ml-example4/.env
    - ml-example4/utils_and_constants.py
    outs:
    - ml-example4/evaluation-result/confusion_matrix.png
    metrics:
 

### Ex.5 - Running Hyperparameter tuning DVC pipelines

In this exercise, you will run the hyperparameter training and model training targets outlined in the dvc.yaml pipeline. The dvc.yaml file outlines the DVC workflow orchestrating the jobs and lists commands, dependencies, and outputs.

You will experiment with running these pipelines independently and observe the interaction between the two via the best parameter configuration file rfc_best_params.json.

In your design, this file is meant to be edited manually for training jobs or, alternatively, as an output of a hyperparameter tuning job.

The dvc.yaml file outlines the DVC workflow orchestrating the hyperparameter tuning and listings commands, dependencies, and outputs.

NOTE: You will start working on the main branch. Git and DVC are already initialized for you.

**Instruction:**
                                                                                    
1. Run DVC training pipeline (target name train) and observe changes in metrics.json (values should be populated).
2. Commit changes with git add . && git commit -m "train on main".
3. Checkout new branch git checkout -b hp_tune_and_train and force run DVC hyperparameter tuning pipeline (target name hp_tune); results file hp_tuning_results.md should now appear in file browser.
4. Run DVC training pipeline again, and compare metrics with main branch using DVC using dvc metrics diff command

------------------
```
dvc repro train
git add . && git commit -m "train on main"
git checkout -b hp_tune_and_train
dvc repro -f hp_tune
dvc repro train
dvc metrics diff
dvc metrics diff main
```

In [93]:
!dvc metrics diff --all

Path                                        Metric     HEAD    workspace    Change
ml-example4\evaluation-result\metrics.json  accuracy   0.9998  0.9998       0.0
ml-example4\evaluation-result\metrics.json  f1_score   0.9996  0.9996       0.0
ml-example4\evaluation-result\metrics.json  precision  1.0     1.0          0.0
ml-example4\evaluation-result\metrics.json  recall     0.9993  0.9993       0.0


## 4.3 GitHub Actions workflow for Hyperparameter Tuning

```
name: comments-ml-example4

on:
  # Only trigger, when `comments-ml-example3` and `comments-ml-example3-v2` workflows succeeded
  workflow_run:
    workflows: 
      - comments-ml-example3
      - comments-ml-example3-v2
    types:
      - completed
  pull_request:
    branches: ["**"]

permissions: write-all

jobs:
  train_and_test_model_ml4:
    runs-on: ubuntu-latest
    steps:
      - name: Checkout
        uses: actions/checkout@v4

      - name: Setup Python
        uses: actions/setup-python@v5
        with:
          python-version: 3.11.9

      - name: Install Dependencies
        run: |
          pip install -r requirements.txt

      - name: Train and test model
        run: |
          python ml-example4/preprocess_dataset.py
          python ml-example4/hp_tuning.py
          python ml-example4/train.py

      - name: Create DVC pipeline
        run: |
          python ml-example4/dvc_pipeline_creation.py

      - name: Setup DVC
        uses: iterative/setup-dvc@v1

      - name: Run DVC pipeline
        run: dvc repro --force
        
      - name: Setup CML GitHub Actions
        uses: iterative/setup-cml@v3

      - name: Write CML report
        env:
          REPO_TOKEN: ${{secrets.GITHUB_TOKEN}}
        run: |
          cat ml-example4/evaluation-result/hp_tuning_results.md >> report4.md
          echo "### Metrics comparison with master branch" >> report4.md
          dvc metrics diff --all --md >> report4.md
          echo "![Confusion Matrix Plot](ml-example4/evaluation-result/confusion_matrix.png)" >> report4.md
          cml comment create report4.md

```

### Ex.6 - Setup Hyperparameter Tuning in GitHub Actions
Imagine a repository with the structure shown in the editor. Your task is to finish the scaffolded .github/workflows/hp_cml.yaml to accomplish the hyperparameter tuning and open a new pull request from a new training branch to main that will run the training pipeline by reading best hyperparameters from rfc_best_params.json.

By convention, hyperparameter tuning branches start with hp_tune/ and training branches start with train/.

Ide Exercise Instruction
100XP
Guard the hyperparameter tuning job so that it only gets triggered from the correct head branch prefix hp_tune/.
Write the correct cml subcommand to create a pull request.
Write the correct prefix for the new head (model training) branch and target branch (where code is merged) in the cml subcommand.
Write the JSON output file from the hyperparameter tuning job in the cml subcommand.

**hp_cml.yaml**

-------------------------------

```
name: hp-tuning

on:
  pull_request:
    branches: main

permissions: write-all

jobs:
  hp_tune:
    # Only run job if the current repository
    # starts with the right prefix
    if: startsWith(github.head_ref, 'hp_tune/')
    runs-on: ubuntu-latest
    steps:
      - name: Checkout 
        uses: actions/checkout@v3
      
      - name: Setup Python
        uses: actions/setup-python@v4
        with:
          python-version: 3.9

      - name: Setup DVC
        uses: iterative/setup-dvc@v1

      - name: Setup CML
        uses: iterative/setup-cml@v1

      - name: Install dependencies
        run: pip install -r requirements.txt

      - name: Run DVC pipeline
        run: |
          dvc repro -f hp_tune
      
      - name: Create training branch
        env:
          REPO_TOKEN: ${{ secrets.GITHUB_TOKEN }}
        run: |
          # Finish the create pull request command
          cml pr create \
          --user-email hp-bot@cicd.ai \
          --user-name HPBot \
          --message "HP tuning" \
          # Write the new head branch name
          --branch train/${{ github.sha }} \
          # Write the target branch name
          --target-branch main \
          # Commit the hyperparameter job output file
          rfc_best_params.json
```

-------------------------------

**train.yaml**

-------------------------------

```
name: train

on:
  pull_request:
    branches: main

permissions: write-all

jobs:
  train_and_publish_report:
    if: startsWith(github.head_ref, 'train/')
    runs-on: ubuntu-latest
    steps:
      - name: Checkout 
        uses: actions/checkout@v3
      
      - name: Setup Python
        uses: actions/setup-python@v4
        with:
          python-version: 3.9

      - name: Setup DVC
        uses: iterative/setup-dvc@v1

      - name: Setup CML
        uses: iterative/setup-cml@v1

      - name: Install dependencies
        run: pip install -r requirements.txt

      - name: Run DVC pipeline
        run: |
          dvc pull
          dvc repro train
      
      - name: Create CML report
        env:
          REPO_TOKEN: ${{ secrets.GITHUB_TOKEN }}
        run: |
          git fetch --prune
          dvc metrics diff --md main >> metrics_compare.md
          cml comment create metrics_compare.md
```

------------------------------